# Named Entity Recognition and Classification 

Evaluación y seleccion de características


## Estructura del modulo de etiquetado

 * `NamedEntityTagger' - entrena el clasificador a partir del dataset de entrenamiento 
 * `tag()` - toma una frase como entrada y propone una secuencia de etiquetas de tipo de entidad 
   * la frase de entrada es una lista de tuplas formados por tokens etiquetados con información moorfosintactica
   * la salida hace zip de la frase y la secuencia de etiquetas de entidad (ne_tag)

## Extracción de características

 * la extraccion de caracteristicas se encapsula en la función `ne_features`
   * `sentence` - lista de tokens etiquetados con tag morfosintactico 
   * `i` - indice para el token que queremos reprensentar en una instancia para el algoritmo de clasificación 
   * `history` - lista de etiquetas de entidad hasta la posicion i-1 

In [ ]:
import nltk 

class NamedEntityTagger(nltk.TaggerI):
    def __init__(self,train_sents):
        train_set=[]
        for sentence in train_sents:
            untagged_sent = [(word, tag) for (word, tag, ne_tag) in sentence]
            history = []
            for i, (word, tag, ne_tag) in enumerate(sentence):
                featureset = ne_features(untagged_sent, i, history)
                train_set.append( (featureset, ne_tag) ) 
                history.append(ne_tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        
        
    def tag(self, sentence):
        history = []
        for i, (word, tag) in enumerate(sentence):
            featureset = ne_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

## Funcion para la extracción de características

In [ ]:
def ne_features(sentence, i, history):
    word, pos = sentence[i]
    return {"word": word}

## Entrenamiento

In [ ]:
from nltk.corpus import conll2002

## Ejemplo de uso que deseamos

# Instanciar el tagger 
# - Usa el conjunto de entrenamiento etiquetado
# - Extraer características
# - Entrenar el modelo
train_sentences = conll2002.iob_sents('esp.train')
nerctagger = NamedEntityTagger(train_sentences)

## Etiquetado de una frase de ejemplo 
#### (index = 107)

In [ ]:
test_sentences = conll2002.iob_sents('esp.testa')
test_sentence = [(word,tag) for (word,tag, ne_tag) in test_sentences[107]]

# Etiquetar oraciones con la tercera columna (ne_tag) dadas las otras dos (word, tag)
# - Extraer características 
# - Usar el modelo entrenado para inferir la nueva etiqueta
# - Otros pasos?
nerctagger.tag(test_sentence)

## Evaluación

In [ ]:
# Ver la secuencia etiquetada por nosotros
tagged_sentence = nerctagger.tag(test_sentence)
[(ne_tag) for (pair,ne_tag) in tagged_sentence]

In [ ]:
# Ver la secuencia etiquetada en el gold standard 
[(ne_tag) for (word,tag, ne_tag) in test_sentences[107]]

In [ ]:
# Ver las dos secuencias pareadas
zip([(ne_tag) for (word,tag, ne_tag) in test_sentences[107]], [(ne_tag) for (pair,ne_tag) in tagged_sentence] )

### Evaluar Precision, Recall y F para una de las etiquetas (I-ORG) y la frase de ejemplo

In [ ]:
import collections 

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (word,tag,label) in enumerate(test_sentences[107]):
        refsets[label].add(i)

for i, (pair,predicted) in enumerate(tagged_sentence):
        testsets[predicted].add(i)

label_type='I-ORG'
print 'precision:', nltk.metrics.precision(refsets[label_type], testsets[label_type])
print 'recall:', nltk.metrics.recall(refsets[label_type], testsets[label_type])
print 'F-measure:', nltk.metrics.f_measure(refsets[label_type], testsets[label_type])        

In [ ]:
for label_type in ['B-LOC','I-LOC','B-ORG','I-ORG','B-PER','I-PER']:
    print 'precision(%s):' % label_type, nltk.metrics.precision(refsets[label_type], testsets[label_type])
    print 'recall(%s):' % label_type, nltk.metrics.recall(refsets[label_type], testsets[label_type])
    print 'F-measure(%s):' % label_type, nltk.metrics.f_measure(refsets[label_type], testsets[label_type])
    print '\n'

### Funcion de evaluación

* Para cada frase del corpus de test compara los etiquetados
* Mide Precision, Recall y F para cada una de los tipos de  etiquetas interesantes - todas salvo O 
* Haz la media (macro-average) para cada una de los tipos de etiqueta

In [ ]:

def eval(nerctagger, test_sentences):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    i = 0
    for test_sentence in test_sentences:
        tagged_sentence = nerctagger.tag([(word,tag) for (word,tag, ne_tag) in test_sentence])
        for ((word,tag,label),(pair,predicted)) in zip(test_sentence,tagged_sentence):
            refsets[label].add(i)
            testsets[predicted].add(i)
            i = i+1

    tags = ['B-LOC','I-LOC','B-ORG','I-ORG','B-PER','I-PER']
    
    (ma_precision, ma_recall, ma_fmeasure) = (0,0,0)
    for label_type in tags:
        precision = nltk.metrics.precision(refsets[label_type], testsets[label_type])
        recall = nltk.metrics.recall(refsets[label_type], testsets[label_type])
        fmeasure = nltk.metrics.f_measure(refsets[label_type], testsets[label_type])
        print 'precision(%s):' % label_type, precision 
        print 'recall(%s):' % label_type, recall 
        print 'F-measure(%s):' % label_type, fmeasure
        print '\n'
        ma_precision += precision
        ma_recall += recall
        ma_fmeasure += fmeasure
        
        
    print "--------------------------------------------------------------------------------"
    print "Precision (Ma):", ma_precision/len(tags)  
    print "Recall (Ma):", ma_recall/len(tags)
    print "F-measure (Ma):", ma_fmeasure/len(tags)
    print "--------------------------------------------------------------------------------"

### Evalua el etiquetados en el conjunto de datos de prueba

In [ ]:
eval(nerctagger, test_sentences)

## Extracción de características - usando la ventana de palabras previa 

In [ ]:

def ne_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    if i == 0:
        prevtag = "<START>"
    else:
        prevtag = history[i-1]        
    return {"word": word, "isAlnum": word.isalnum(), "isDigit": word.isdigit(), "pos": pos, "prevword": prevword, "prevpos": prevpos, "prevtag" : prevtag}

nerctagger = NamedEntityTagger(train_sentences)
eval(nerctagger, test_sentences)